In [117]:
# This file includes concepts related to creating custom function and using them as tools

In [118]:
# Importing the libraries
from google.adk.agents import LlmAgent
from google.adk.tools import ToolContext, AgentTool
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.code_executors import BuiltInCodeExecutor
from dotenv import load_dotenv
import os

In [119]:
# Configuration check
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print("API KEY FOUND")
else: 
    print("CONFIGURATION ERROR")
    

API KEY FOUND


In [120]:
# Retry configuration
def create_retry_config():
    '''
    This function creates retry configuration for the agent
    '''
    return types.HttpRetryOptions(
        attempts=5,
        exp_base=2,
        initial_delay=2,
        http_status_codes=[429, 500, 501, 503, 504]
    )

retry_config = create_retry_config()

In [121]:
# The agent built here is a Currency Convertor Agent, which takes currency from one denomination and converts it to another, and uses function to calculate the conversion fess involved

# Defining a custom mock function that gets the fees applicable for a specific payment method

def get_fee_for_payment_method(method: str) -> dict:
    # This acts as an instruction for the llm, of what is this tool, how it functions and what to expect as an output
    '''Looks up the transaction fee percentage for a given payment method.

    This tool simulates the looking up a company's internal fee structure based on the name
    of the payment method provided by the user.

    Args: 
        method: The name of the payment method. It should be descriptive.
        e.g., "platinum credit card" or "bank transfer"

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": Payment method not found}
    '''

    # This dictionary simulates looking up for company's internal fee structure on the basis of payment method
    fee_database = {
        "platinum credit card": 0.02,
        "gold debit card": 0.035,
        "bank transfer": 0.01
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {"status": "error", "error_message": f"Payment method '{method}' not found"}
    
print("Fee lookup function created")

Fee lookup function created


In [122]:
# This function looks up for exchange rate between two target currencies
def get_exchange_rate(base_currency: str, target_currency: str)-> dict:
    '''Looks for the conversion rate between two target currencies

    This tool looks up for the conversion rate between the two currencies from the company's database
    or calls to an external api if conversion rate is not found in the database.

    Args:
        base_currency: The ISO 4217 currency code of the currency you are
        converting from (e.g., "USD")

        target_currency: The ISO 4217 currency code of the currency you are
        converting to (e.g., "EUR")

    Returns: 
        Dictionary with status and rate information
        Success: {"status": "success", "rate": 0.94}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    '''

    # Simulating a static data
    # In real world production scenario, this function would internally make an api call with base and target currency to fetch the latest conversion rate

    rate_database = {
        "usd": {
            "eur": 0.94,
            "jpy": 133.90,
            "inr": 91.88
        }
    }
    # Validation and processing
    base = base_currency.lower().strip()
    target = target_currency.lower().strip()

    # Return result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {"status": "error", "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}"}
    

print("Exchange rate function created")
    

Exchange rate function created


In [123]:
# Currency agent with access to custom function tools defined above
currency_agent=LlmAgent(
    name="currency_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''You are a smart currency conversion assistant.

    For currency conversion request:
    1. Use `get_fee_for_payment_method()` to find the transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check for "status" field in the tool response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` and give complete breakdown
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amount. Your explaination must include
        the fee processing and its value in original currency, the amount remaining after the fee, and the exchange rate
        used for final conversion.

    If any tool returns error, explain the issue to the user clearly
    ''',
    tools=[get_fee_for_payment_method, get_exchange_rate]
)
print("Currency agent created")

Currency agent created


In [124]:
# currency_runner = InMemoryRunner(agent=currency_agent)
# _ = await currency_runner.run_debug(
#     "I want to convert 500 US Dollors to Indian Rupees using my bank transfer. How much amount will I receive?"
# )

In [125]:
# This is an calculation agent, that takes BuiltInCodeExecutor to execute python code
calculation_agent= LlmAgent(
    name="CalculationAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=''' You are a specialized calculator that ONLY responds with Python code.
    You are forbidden from providing any text, explanations, or conversational responses.

    Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.

    **RULES**
    1. Your output MUST be ONLY a Python code block.
    2. Do NOT write any text before or after the code block.
    3. The Python code MUST calculate the result.
    4. The Python code MUST print the final result to stdout.
    5. You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.

    Failure to follow these rules will result in an error.

    ''',
    code_executor=BuiltInCodeExecutor() # Use the built-in Code Executor Tool.
)

In [126]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction='''You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.
    For any currency conversion request:

    1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
    2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
    3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
    4. Calculate Final Amount (CRITICAL): You are strictly prohobited from performing any arithemetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final comverted amount.
       This code will use the fee information from step 1 and exchange rate from step 2.
    5. Provide Detailed Breakdown: In your summary, you must:
        * State the final converted amount.
        * Explain how the result was calculated, including:
            * The fee percentage and fee amount in original currency
            * The amount remaining after deducting the fee
            * The exchange rate applied.

    ''',
    tools=[
        get_fee_for_payment_method, 
        get_exchange_rate,
        AgentTool(agent=calculation_agent)
    ]
)

print("Enhanced currency agent created")

Enhanced currency agent created


In [127]:
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".


In [129]:
response = await enhanced_runner.run_debug(
    "Convert 19960 USD to INR using bank transfer. Show me the precise calculation"
)


 ### Continue session: debug_session_id

User > Convert 19960 USD to INR using bank transfer. Show me the precise calculation


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/arpit/devlopement/agentic-ai-google-kaggle/myvenv/lib/python3.9/site-packages/google/adk/agents", which implies app name "agents".


enhanced_currency_agent > Here's the breakdown for converting 19960 USD to INR using a bank transfer:

*   **Amount to convert:** 19960 USD
*   **Payment Method:** Bank Transfer
*   **Fee:** 1.0% (which is 199.60 USD)
*   **Amount after fee:** 19760.40 USD
*   **Exchange Rate:** 1 USD = 91.88 INR
*   **Final Amount in INR:** 1815557.51 INR

The final amount you will receive in INR is 1815557.51 INR.
